In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
labelEncoder = LabelEncoder()

In [2]:
bond_ratings = pd.read_csv('bond_ratings.csv')
## "tag"
bond_ratings = bond_ratings.drop(['us_govt_bond_rating'], axis = 1)

fund_allocations = pd.read_csv('fund_allocations.csv')
## "id" change name
fund_allocations['tag'] = fund_allocations['id']
fund_allocations = fund_allocations.drop(['id'], axis = 1)

fund_ratios = pd.read_csv('fund_ratios.csv')
## "tag"

fund_config = pd.read_csv('fund_config.csv')
## "fund_id" merge

fund_specs = pd.read_csv('fund_specs.csv')
## "tag"

other_specs = pd.read_csv('other_specs.csv')
other_specs = other_specs.drop(['pc_ratio', 'pe_ratio', 'ps_ratio', 'mmc', 'greatstone_rating', 'pb_ratio', 'fund_return_3years'], axis = 1)
## "tag"

return_3year = pd.read_csv('return_3year.csv')
## "tag"

return_5year = pd.read_csv('return_5year.csv')
## "tag"

return_10year = pd.read_csv('return_10year.csv')
## "fund_Id" merge

In [3]:
### order the data correctly by merging all the tables based on common columns
combine = pd.merge(bond_ratings, fund_allocations,how='inner',on="tag")
combine = pd.merge(combine, fund_ratios, how='inner', on="tag")
combine = pd.merge(combine, fund_config, how='inner', on="fund_id")
combine = pd.merge(combine, fund_specs, how='inner', on="tag")
combine = pd.merge(combine, other_specs, how='inner', on="tag")
combine = pd.merge(combine, return_3year, how='inner', on="tag")
combine = pd.merge(combine, return_5year, how='inner', on="tag")
combine = pd.merge(combine, return_10year, how='inner', on="fund_id")

### Save the order of the tag and fund_id
fundIdTag = pd.DataFrame()
fundIdTag['tag'] = combine['tag']
fundIdTag['fund_id'] = combine['fund_id']
fundIdTag['greatstone_rating'] = combine['greatstone_rating']

###seperate into object, floats and ints so pre-processing is simpler.
combineFloat = pd.DataFrame()
combineInt = pd.DataFrame()
combineObj = pd.DataFrame()
for col in combine:
    if(combine[col].dtype == 'float64'):
        combineFloat[col] = combine[col]
    elif (combine[col].dtype == 'int64'):
        combineInt[col] = combine[col]
    else:
        combineObj[col] = combine[col]

In [4]:
temp = combineObj.drop(['fund_id', 'fund_name', 'currency'], axis = 1)
labelObj = pd.DataFrame()
###using Mode replacement strategy
for col in temp:
    if len(temp[col].unique()) > len(temp['category'].unique()) and col != 'inception_date' and col != 'parent_company':
        combineFloat[col] = temp[col].astype('str')
        combineFloat[col] = combineFloat[col].str.replace(',', '')
        combineFloat.loc[combineFloat[col] == "nan", col] = np.NaN
        combineFloat[col] = pd.to_numeric(combineFloat[col])
        mode = combineFloat[col].mode()
        combineFloat[col].fillna(mode, inplace = True)
        combineFloat.loc[combineFloat[col] == np.NaN, col] = mode
        combineObj = combineObj.drop([col], axis = 1)
        
    elif col != 'inception_date':
        labelObj[col] = temp[col].astype('str')
        mode = labelObj[col].mode()
        labelObj[col].fillna(mode, inplace = True)
        labelObj.loc[labelObj[col] == np.NaN, col] = mode
        labelObj[col] = labelEncoder.fit_transform(labelObj[col])
        
    else:
        combineFloat[col] = pd.to_datetime(temp[col])
        combineFloat[col] = pd.to_numeric(combineFloat[col])
        combineObj = combineObj.drop([col], axis = 1)

In [5]:
bins = KBinsDiscretizer(n_bins=100, encode='ordinal')
for col in combineFloat:
    median = combineFloat[col].median()
    combineFloat[col].fillna(median, inplace = True)
    combineFloat.loc[combineFloat[col] == np.NaN, col] = median
binnedFloatsArray = bins.fit_transform(combineFloat)
binnedFloats = pd.DataFrame(binnedFloatsArray, columns=combineFloat.columns)

C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are re

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.svm import SVC

In [77]:
submit = pd.read_csv('sample_submission.csv');

binnedFloats['fund_id'] = fundIdTag['fund_id']
binnedFloats['greatstone_rating'] = fundIdTag['greatstone_rating']
train = binnedFloats.dropna()
X = train.drop(['greatstone_rating', 'fund_id'], axis = 1)
Y = train['greatstone_rating']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [78]:
rfcl = RandomForestClassifier(n_estimators = 100, n_jobs=2, random_state = 37, criterion='entropy')
rfcl = rfcl.fit(x_train, y_train)
y_predict0 = rfcl.predict(x_test)
print('Classification: \n', metrics.classification_report(y_test, y_predict0))

Classification: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       146
         1.0       0.82      0.69      0.75       133
         2.0       0.77      0.77      0.77       408
         3.0       0.78      0.85      0.81       676
         4.0       0.80      0.78      0.79       475
         5.0       0.84      0.72      0.77       162

    accuracy                           0.81      2000
   macro avg       0.83      0.80      0.82      2000
weighted avg       0.81      0.81      0.80      2000



In [83]:
clf = SVC(gamma=0.1)
clf.fit(x_train, y_train)
y_predict = clf.predict(x_test).astype('int')
print('Classification: \n', metrics.classification_report(y_test, y_predict))

Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.10      0.19       146
         1.0       1.00      0.01      0.01       133
         2.0       0.88      0.02      0.03       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.03      0.06       475
         5.0       1.00      0.01      0.01       162

    accuracy                           0.36      2000
   macro avg       0.87      0.19      0.14      2000
weighted avg       0.75      0.36      0.21      2000



In [89]:
for i in range(5,30):
    print("\nGamma: ", i/100)
    clf = SVC(gamma=i/100)
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test).astype('int')
    print('Classification: \n', metrics.classification_report(y_test, y_predict))


Gamma:  0.05
Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.13      0.23       146
         1.0       1.00      0.02      0.03       133
         2.0       0.93      0.03      0.06       408
         3.0       0.35      1.00      0.52       676
         4.0       0.97      0.07      0.14       475
         5.0       1.00      0.02      0.05       162

    accuracy                           0.37      2000
   macro avg       0.88      0.21      0.17      2000
weighted avg       0.76      0.37      0.24      2000


Gamma:  0.06
Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.12      0.21       146
         1.0       1.00      0.01      0.01       133
         2.0       0.92      0.03      0.06       408
         3.0       0.35      1.00      0.52       676
         4.0       0.97      0.07      0.12       475
         5.0       1.00      0.02      0.04       162

    accuracy 

C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.07      0.13       146
         1.0       0.00      0.00      0.00       133
         2.0       0.88      0.02      0.03       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.02      0.04       475
         5.0       1.00      0.01      0.01       162

    accuracy                           0.35      2000
   macro avg       0.70      0.18      0.12      2000
weighted avg       0.69      0.35      0.20      2000


Gamma:  0.13


C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.06      0.12       146
         1.0       0.00      0.00      0.00       133
         2.0       0.86      0.01      0.03       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.02      0.03       475
         5.0       1.00      0.01      0.01       162

    accuracy                           0.35      2000
   macro avg       0.70      0.18      0.12      2000
weighted avg       0.68      0.35      0.20      2000


Gamma:  0.14


...

C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.04      0.08       146
         1.0       0.00      0.00      0.00       133
         2.0       0.00      0.00      0.00       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.00      0.01       475
         5.0       1.00      0.01      0.01       162

    accuracy                           0.34      2000
   macro avg       0.56      0.18      0.10      2000
weighted avg       0.51      0.34      0.18      2000


Gamma:  0.28


C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.04      0.08       146
         1.0       0.00      0.00      0.00       133
         2.0       0.00      0.00      0.00       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.00      0.01       475
         5.0       0.00      0.00      0.00       162

    accuracy                           0.34      2000
   macro avg       0.39      0.17      0.10      2000
weighted avg       0.43      0.34      0.18      2000


Gamma:  0.29
Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.04      0.08       146
         1.0       0.00      0.00      0.00       133
         2.0       0.00      0.00      0.00       408
         3.0       0.34      1.00      0.51       676
         4.0       1.00      0.00      0.01       475
         5.0       0.00      0.00      0.00       162

    accuracy               

C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [82]:
temp = y_predict
temp[y_predict0 == 0] = 0

print('Classification: \n', metrics.classification_report(y_test, temp))

Classification: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       146
         1.0       0.00      0.00      0.00       133
         2.0       1.00      0.01      0.02       408
         3.0       0.37      1.00      0.54       676
         4.0       1.00      0.01      0.02       475
         5.0       1.00      0.01      0.01       162

    accuracy                           0.42      2000
   macro avg       0.73      0.34      0.26      2000
weighted avg       0.72      0.42      0.26      2000



In [76]:
predictions = pd.DataFrame()
predictions['temp'] = temp
predictions.loc[predictions.query('temp == 5').sample(frac=.9).index, 'temp'] = 3
predictions.loc[predictions.query('temp == 4').sample(frac=.9).index, 'temp'] = 3
predictions.loc[predictions.query('temp == 2').sample(frac=.9).index, 'temp'] = 3

print('Classification: \n', metrics.classification_report(y_test, predictions['temp']))

Classification: 
               precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       285
         1.0       1.00      0.07      0.12       259
         2.0       1.00      0.01      0.03       848
         3.0       0.37      1.00      0.54      1375
         4.0       1.00      0.02      0.03       914
         5.0       1.00      0.01      0.02       319

    accuracy                           0.42      4000
   macro avg       0.90      0.35      0.29      4000
weighted avg       0.78      0.42      0.28      4000



# Final

In [7]:
ind = 1

In [9]:
submit = pd.read_csv('sample_submission.csv');

binnedFloats['fund_id'] = fundIdTag['fund_id']
binnedFloats['greatstone_rating'] = fundIdTag['greatstone_rating']

train = binnedFloats.dropna()

test = binnedFloats[binnedFloats['greatstone_rating'].isna()]
test = pd.merge(submit, test, how="inner", on="fund_id")

x_train = train.drop(['greatstone_rating', 'fund_id'], axis = 1)
y_train = train['greatstone_rating']

x_test = test.drop(['greatstone_rating_x', 'greatstone_rating_y', 'fund_id'], axis = 1)

rfcl = RandomForestClassifier(n_estimators = 100, random_state = 37, criterion='entropy')
rfcl = rfcl.fit(x_train, y_train)
print(rfcl.score(x_train , y_train))
y_predict0 = rfcl.predict(x_test)

clf = SVC()
clf.fit(x_train, y_train)

print(clf.score(x_train , y_train))

y_predict = clf.predict(x_test).astype('int')

predict = pd.DataFrame()
predict['predict'] = y_predict.astype('int')

1.0


C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9973


In [51]:
submit['greatstone_rating'] = 3;
submit['greatstone_rating'][y_predict0 == 0] = 0

cond = predict.predict.values == 1
equals = np.cumsum(cond)
idx = np.searchsorted(equals, 8)
submit['greatstone_rating'][idx] = 1

cond = predict.predict.values == 2
equals = np.cumsum(cond)
idx = np.searchsorted(equals, 1)
submit['greatstone_rating'][idx] = 2

cond = predict.predict.values == 4
equals = np.cumsum(cond)
idx = np.searchsorted(equals, 2)
submit['greatstone_rating'][idx] = 4

cond = predict.predict.values == 5
equals = np.cumsum(cond)
idx = np.searchsorted(equals, 4)
submit['greatstone_rating'][idx] = 5

submit.to_csv('TEST_Final.csv', index=False)

C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Tim Buckley\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Tim Buck

91